In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm, uniform
from kan import *
import torch
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import scipy as sp

In [ ]:
plt.rcParams.update({
    'font.size': 12,               # Font size
    'axes.labelsize': 15,          # Label size
    'axes.titlesize': 15,          # Title size
    'axes.linewidth': 1.2,         # Axis line width
    'xtick.labelsize': 10,         # Size of x-axis tick labels
    'ytick.labelsize': 10,         # Size of y-axis tick labels
    'xtick.major.size': 6,         # Length of major ticks on the x-axis
    'ytick.major.size': 6,         # Length of major ticks on the y-axis
    'xtick.major.width': 1.0,      # Width of major ticks on the x-axis
    'ytick.major.width': 1.0,      # Width of major ticks on the y-axis
    'figure.dpi': 300,             # Image resolution
    'savefig.dpi': 300,            # Resolution for saving figures
    'figure.figsize': (12, 5),     # Figure size
})

In [ ]:
import clayton
from clayton.rng import base, evd, archimedean, monte_carlo
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm, expon

In [ ]:
n_samples, theta = 2500, 1
copula = archimedean.Clayton(theta=theta, n_samples=n_samples)
sample = copula.sample_unimargin()
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(sample[:,0], sample[:,1], edgecolors='#6F6F6F', color='#C5C5C5', s=4)
ax.set_xlabel(r'$x_1$', fontsize=16)
ax.set_ylabel(r'$x_2$', fontsize=16)
ax.set_title(r'$\theta = 1$') 
plt.show()

In [ ]:
def MC_B1(M, n, theta):
    x_min, x_max = 0, 1
    y_min, y_max = 0, 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))

    u_quantile = 0.95
    probabilities_all = [] 
    
    for i in range(M):
        np.random.seed(i) 
        
        # Clayton Copula
        copula = archimedean.Clayton(theta=theta, n_samples=n)
        sample = copula.sample_unimargin()
        X1 = sample[:,0]
        X2 = sample[:,1]

        # Y_X ~ Unit Fréchet
        U = np.random.uniform(0, 1, size=n)
        Y_X = 1 / -np.log(n / (n + 1) * U)

        u = np.quantile(Y_X, u_quantile)
        mean = [np.exp(-u), np.exp(-u)]
        cov = [[1, 0], [0, 1]] 
        mvnorm = multivariate_normal(mean, cov)
        Phi_X = np.array([mvnorm.cdf([x1, x2]) for x1, x2 in zip(X1, X2)])
        mask = Y_X > u
        I = np.zeros(n, dtype=int)
        I[mask] = np.random.binomial(1, Phi_X[mask])

        # Dataframe
        df = pd.DataFrame({'X1': X1, 'X2': X2, 'Y': Y_X, 'I': I})
        df = df.sort_values(by=['X1', 'X2'], ascending=[True, True])
        df_sim = df[df['Y'] > u]

        # Neural inputs
        x_train_2d = np.zeros((df_sim.shape[0], 2))

        x_train_2d[:, 0] = df_sim['X1']
        x_train_2d[:, 1] = df_sim['X2']
        y_train_2d = df_sim['I'].values

        x_train_tensor = torch.tensor(x_train_2d, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train_2d, dtype=torch.long)

        model = KAN(width=[2, 2], grid=2, k=3)

        dataset = {
            'train_input': x_train_tensor,
            'train_label': y_train_tensor,
            'test_input': x_train_tensor,
            'test_label': y_train_tensor
        }

        model.fit(dataset, opt="LBFGS", steps=100, loss_fn=torch.nn.CrossEntropyLoss())

        grid_tensor = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)

        with torch.no_grad():
            Z = model(grid_tensor)
            Z = torch.softmax(Z, dim=1)
            probabilities = Z[:, 1].reshape(xx.shape)

        probabilities_all.append(probabilities)

    return probabilities_all, xx, yy

In [ ]:
probabilities_all_1_1, xx, yy = MC_B1(500, 2500, 1)

In [ ]:
probabilities_all_1_5, xx, yy = MC_B1(500, 2500, 5)

In [ ]:
probabilities_all_1_10, xx, yy = MC_B1(500, 2500, 10)

In [ ]:
def bivariate_cdf(x, y, mean, cov):
    pos = np.dstack((x, y))
    return multivariate_normal.cdf(pos, mean=mean, cov=cov)

probabilities_all_1_1 = np.array(probabilities_all_1_1)
monte_carlo_mean_surface_1_1 = np.mean(probabilities_all_1_1, axis=0)

probabilities_all_1_5 = np.array(probabilities_all_1_5)
monte_carlo_mean_surface_1_5 = np.mean(probabilities_all_1_5, axis=0)

probabilities_all_1_10 = np.array(probabilities_all_1_10)
monte_carlo_mean_surface_1_10 = np.mean(probabilities_all_1_10, axis=0)

cdf_values = bivariate_cdf(xx, yy, [0, 0], [[1, 0], [0, 1]] )

In [ ]:
fig = plt.figure(figsize=(12, 5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.3)

ax1 = fig.add_subplot(gs[0])
im1 = ax1.pcolor(xx, yy, monte_carlo_mean_surface_1_1, shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax1.set_xlabel(r'$x_1$', fontsize=16)
ax1.set_ylabel(r'$x_2$', fontsize=16)
ax1.set_title(r'MC Mean KANE POC Surface ($\theta = 1$)')

ax2 = fig.add_subplot(gs[1])
im2 = ax2.pcolor(xx, yy, monte_carlo_mean_surface_1_5, shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax2.set_xlabel(r'$x_1$', fontsize=16)
ax2.set_ylabel(r'$x_2$', fontsize=16)
ax2.set_title(r'MC Mean KANE POC Surface ($\theta = 5$)')

cbar_ax = fig.add_subplot(gs[2])
fig.colorbar(im2, cax=cbar_ax)

pos = cbar_ax.get_position()  
cbar_ax.set_position([pos.x0 - 0.02, pos.y0, pos.width, pos.height]) 

plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.3)

ax1 = fig.add_subplot(gs[0])
im1 = ax1.pcolor(xx, yy, monte_carlo_mean_surface_1_10, shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax1.set_xlabel(r'$x_1$', fontsize=16)
ax1.set_ylabel(r'$x_2$', fontsize=16)
ax1.set_title(r'MC Mean KANE POC Surface ($\theta = 10$)')

ax2 = fig.add_subplot(gs[1])
im2 = ax2.pcolor(xx, yy, cdf_values, shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax2.set_xlabel(r'$x_1$', fontsize=16)
ax2.set_ylabel(r'$x_2$', fontsize=16)
ax2.set_title("True POC Surface")

cbar_ax = fig.add_subplot(gs[2])
fig.colorbar(im2, cax=cbar_ax)

pos = cbar_ax.get_position()  
cbar_ax.set_position([pos.x0 - 0.02, pos.y0, pos.width, pos.height]) 

plt.tight_layout()
plt.show()

In [ ]:
def MC_B2(M, n, theta):
    def f(x, y):
        return 0.5 + 0.4 *np.exp(- x) * np.cos(2 * np.pi * y) + 1/u**2

    x_min, x_max = 0, 1
    y_min, y_max = 0, 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))

    u_quantile = 0.95
    probabilities_all = [] 
    
    for i in range(M):
        np.random.seed(i) 
        
        # Clayton Copula
        copula = archimedean.Clayton(theta=theta, n_samples=n)
        sample = copula.sample_unimargin()
        X1 = sample[:,0]
        X2 = sample[:,1]

        # Y_X ~ Unit Fréchet
        U = np.random.uniform(0, 1, size=n)
        Y_X = 1 / -np.log(n / (n + 1) * U)
        
        u = np.quantile(Y_X, u_quantile)
        f_X = np.array([f(x1, x2) for x1, x2 in zip(X1, X2)])
        mask = Y_X > u
        I = np.zeros(n, dtype=int)
        I[mask] = np.random.binomial(1, f_X[mask])

        # Dataframe
        df = pd.DataFrame({'X1': X1, 'X2': X2, 'Y': Y_X, 'I': I})
        df = df.sort_values(by=['X1', 'X2'], ascending=[True, True])
        df_sim = df[df['Y'] > u]

        # Neural inputs
        x_train_2d = np.zeros((df_sim.shape[0], 2))

        x_train_2d[:, 0] = df_sim['X1']
        x_train_2d[:, 1] = df_sim['X2']
        y_train_2d = df_sim['I'].values

        x_train_tensor = torch.tensor(x_train_2d, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train_2d, dtype=torch.long)

        model = KAN(width=[2, 2], grid=2, k=3)

        dataset = {
            'train_input': x_train_tensor,
            'train_label': y_train_tensor,
            'test_input': x_train_tensor,
            'test_label': y_train_tensor
        }

        model.fit(dataset, opt="LBFGS", steps=100, loss_fn=torch.nn.CrossEntropyLoss())

        grid_tensor = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)

        with torch.no_grad():
            Z = model(grid_tensor)
            Z = torch.softmax(Z, dim=1)
            probabilities = Z[:, 1].reshape(xx.shape)

        probabilities_all.append(probabilities)

    return probabilities_all, xx, yy

In [ ]:
probabilities_all_2_1, xx, yy = MC_B2(500, 2500, 1)

In [ ]:
probabilities_all_2_5, xx, yy = MC_B2(500, 2500, 5)

In [ ]:
probabilities_all_2_10, xx, yy = MC_B2(500, 2500, 10)

In [ ]:
def f(x, y):
    return 0.5 + 0.4 * np.exp(- x) * np.cos(2 * np.pi * y)

probabilities_all_2_1 = np.array(probabilities_all_2_1)
monte_carlo_mean_surface_2_1 = np.mean(probabilities_all_2_1, axis=0)

probabilities_all_2_5 = np.array(probabilities_all_2_5)
monte_carlo_mean_surface_2_5 = np.mean(probabilities_all_2_5, axis=0)

probabilities_all_2_10 = np.array(probabilities_all_2_10)
monte_carlo_mean_surface_2_10 = np.mean(probabilities_all_2_10, axis=0)

f_values = f(xx, yy)

In [ ]:
fig = plt.figure(figsize=(12, 5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.3)

ax1 = fig.add_subplot(gs[0])
im1 = ax1.pcolor(xx, yy, monte_carlo_mean_surface_2_1, shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax1.set_xlabel(r'$x_1$', fontsize=16)
ax1.set_ylabel(r'$x_2$', fontsize=16)
ax1.set_title(r'MC Mean KANE POC Surface ($\theta = 1$)')

ax2 = fig.add_subplot(gs[1])
im2 = ax2.pcolor(xx, yy, monte_carlo_mean_surface_2_5, shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax2.set_xlabel(r'$x_1$', fontsize=16)
ax2.set_ylabel(r'$x_2$', fontsize=16)
ax2.set_title(r'MC Mean KANE POC Surface ($\theta = 5$)')

cbar_ax = fig.add_subplot(gs[2])
fig.colorbar(im2, cax=cbar_ax)

pos = cbar_ax.get_position()  
cbar_ax.set_position([pos.x0 - 0.02, pos.y0, pos.width, pos.height]) 

plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.3)

ax1 = fig.add_subplot(gs[0])
im1 = ax1.pcolor(xx, yy, monte_carlo_mean_surface_2_10, shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax1.set_xlabel(r'$x_1$', fontsize=16)
ax1.set_ylabel(r'$x_2$', fontsize=16)
ax1.set_title(r'MC Mean KANE POC Surface ($\theta = 10$)')

ax2 = fig.add_subplot(gs[1])
im2 = ax2.pcolor(xx, yy, f_values, shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax2.set_xlabel(r'$x_1$', fontsize=16)
ax2.set_ylabel(r'$x_2$', fontsize=16)
ax2.set_title('True POC Surface')

cbar_ax = fig.add_subplot(gs[2])
fig.colorbar(im2, cax=cbar_ax)

pos = cbar_ax.get_position()  
cbar_ax.set_position([pos.x0 - 0.02, pos.y0, pos.width, pos.height]) 

plt.suptitle('Scenario C', fontsize=18, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
def MC_C(M, n, theta):
    def m1(x1, x2):
        mean = [np.exp(-u), np.exp(-u)] 
        cov = [[1, 0], [0, 1]]
        return multivariate_normal.cdf([x1, x2], mean=mean, cov=cov)

    def m2(x1, x2):
        return 0.5 + 0.4 *np.exp(- x1) * np.cos(2 * np.pi * x2) + 1/u**2

    def m3(x1, x2):
        return 0.8 * x2 * np.sin(np.pi * x1)**2 + 1/(u**2)
    
    x_min, x_max = 0, 1
    y_min, y_max = 0, 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))

    u_quantile = 0.95
    probabilities_all = [] 

    for i in range(M):
        np.random.seed(i)
        
        # Clayton Copula
        copula = archimedean.Clayton(theta=theta, n_samples=n)
        sample = copula.sample_unimargin()
        X1 = sample[:,0]
        X2 = sample[:,1]

        # Y_X ~ Unit Fréchet
        U = np.random.uniform(0, 1, size=n)
        Y_X = 1 / -np.log(n / (n + 1) * U)

        u = np.quantile(Y_X, u_quantile)
        mask = Y_X > u

        probs = np.zeros((n, 3))
        probs[:, 0] = np.array([m1(x1, x2) for x1, x2 in zip(X1, X2)])
        probs[:, 1] = np.array([m2(x1, x2) for x1, x2 in zip(X1, X2)])
        probs[:, 2] = np.array([m3(x1, x2) for x1, x2 in zip(X1, X2)])

        probs_sum = probs.sum(axis=1)
        probs = probs / probs_sum[:, np.newaxis]

        I = np.zeros(n, dtype=int)
        I[mask] = [np.random.multinomial(1, pvals).argmax() for pvals in probs[mask]]

        df = pd.DataFrame({'X1': X1, 'X2': X2, 'Y': Y_X, 'I': I})
        df = df.sort_values(by=['X1', 'X2'], ascending=[True, True])
        df_sim = df[df['Y'] > u]

        x_train_tensor = torch.tensor(df_sim[['X1', 'X2']].values, dtype=torch.float32)
        y_train_tensor = torch.tensor(df_sim['I'].values, dtype=torch.long)

        model = KAN(width=[2, 3], grid=2, k=3)
        dataset = {
            'train_input': x_train_tensor,
            'train_label': y_train_tensor,
            'test_input': x_train_tensor,
            'test_label': y_train_tensor
        }

        model.fit(dataset, opt="LBFGS", steps=100, loss_fn=torch.nn.CrossEntropyLoss())

        grid_tensor = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)
        with torch.no_grad():
            Z = model(grid_tensor)
            Z = torch.softmax(Z, dim=1)
            probabilities = Z.reshape(100, 100, 3)

        probabilities_all.append(probabilities)

    return probabilities_all, xx, yy

In [ ]:
probabilities_all_3_1, xx, yy = MC_C(500, 2500, 1)

In [ ]:
probabilities_all_3_5, xx, yy = MC_C(500, 2500, 5)

In [ ]:
probabilities_all_3_10, xx, yy = MC_C(500, 2500, 10)

In [ ]:
def m1(x1, x2):
    return multivariate_normal.cdf([x1, x2], mean=[0,0], cov=[[1, 0], [0, 1]])

def m2(x1, x2):
    return 0.5 + 0.4 * np.exp(- x1) * np.cos(2 * np.pi * x2)

def m3(x1, x2):
    return 0.8 * x2 * np.sin(np.pi * x1)**2

probabilities_all_3_1 = np.array(probabilities_all_3_1)
monte_carlo_mean_surface_3_1 = np.mean(probabilities_all_3_1, axis=0)

probabilities_all_3_5 = np.array(probabilities_all_3_5)
monte_carlo_mean_surface_3_5 = np.mean(probabilities_all_3_5, axis=0)

probabilities_all_3_10 = np.array(probabilities_all_3_10)
monte_carlo_mean_surface_3_10 = np.mean(probabilities_all_3_10, axis=0)

f_values = np.zeros((10000, 3)) 
f_values[:, 0] = np.array([m1(x1, x2) for x1, x2 in zip(xx.ravel(), yy.ravel())])
f_values[:, 1] = np.array([m2(x1, x2) for x1, x2 in zip(xx.ravel(), yy.ravel())])
f_values[:, 2] = np.array([m3(x1, x2) for x1, x2 in zip(xx.ravel(), yy.ravel())])

f_values_sum = f_values.sum(axis=1)
f_values = f_values / f_values_sum[:, np.newaxis]

f_values = f_values.reshape(100, 100, 3)

In [ ]:
fig = plt.figure(figsize=(12, 5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.3)

ax1 = fig.add_subplot(gs[0])
im1 = ax1.pcolor(xx, yy, monte_carlo_mean_surface_3_1[:, :, 0], shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax1.set_xlabel(r'$x_1$', fontsize=16)
ax1.set_ylabel(r'$x_2$', fontsize=16)
ax1.set_title(r'MC Mean KANE POC Surface ($j = 1,\,\theta = 1)$')

ax2 = fig.add_subplot(gs[1])
im2 = ax2.pcolor(xx, yy, monte_carlo_mean_surface_3_5[:, :, 0], shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax2.set_xlabel(r'$x_1$', fontsize=16)
ax2.set_ylabel(r'$x_2$', fontsize=16)
ax2.set_title(r'MC Mean KANE POC Surface ($j = 1,\,\theta = 5)$')

cbar_ax = fig.add_subplot(gs[2])
fig.colorbar(im2, cax=cbar_ax)

pos = cbar_ax.get_position() 
cbar_ax.set_position([pos.x0 - 0.02, pos.y0, pos.width, pos.height]) 
    
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.3)
    
ax1 = fig.add_subplot(gs[0])
im1 = ax1.pcolor(xx, yy, monte_carlo_mean_surface_3_10[:, :, 0], shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax1.set_xlabel(r'$x_1$', fontsize=16)
ax1.set_ylabel(r'$x_2$', fontsize=16)
ax1.set_title(r'MC Mean KANE POC Surface ($j = 1,\,\theta = 10)$')
    
ax2 = fig.add_subplot(gs[1])
im2 = ax2.pcolor(xx, yy, f_values[:, :, 0], shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax2.set_xlabel(r'$x_1$', fontsize=16)
ax2.set_ylabel(r'$x_2$', fontsize=16)
ax2.set_title(r'True POC Surface $j = 1$')

cbar_ax = fig.add_subplot(gs[2])
fig.colorbar(im2, cax=cbar_ax)

pos = cbar_ax.get_position() 
cbar_ax.set_position([pos.x0 - 0.02, pos.y0, pos.width, pos.height]) 
    
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.3)

ax1 = fig.add_subplot(gs[0])
im1 = ax1.pcolor(xx, yy, monte_carlo_mean_surface_3_1[:, :, 1], shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax1.set_xlabel(r'$x_1$', fontsize=16)
ax1.set_ylabel(r'$x_2$', fontsize=16)
ax1.set_title(r'MC Mean KANE POC Surface ($j = 2,\,\theta = 1)$')

ax2 = fig.add_subplot(gs[1])
im2 = ax2.pcolor(xx, yy, monte_carlo_mean_surface_3_5[:, :, 1], shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax2.set_xlabel(r'$x_1$', fontsize=16)
ax2.set_ylabel(r'$x_2$', fontsize=16)
ax2.set_title(r'MC Mean KANE POC Surface ($j = 2,\,\theta = 5)$')

cbar_ax = fig.add_subplot(gs[2])
fig.colorbar(im2, cax=cbar_ax)

pos = cbar_ax.get_position() 
cbar_ax.set_position([pos.x0 - 0.02, pos.y0, pos.width, pos.height]) 
    
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.3)
    
ax1 = fig.add_subplot(gs[0])
im1 = ax1.pcolor(xx, yy, monte_carlo_mean_surface_3_10[:, :, 1], shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax1.set_xlabel(r'$x_1$', fontsize=16)
ax1.set_ylabel(r'$x_2$', fontsize=16)
ax1.set_title(r'MC Mean KANE POC Surface ($j = 2,\,\theta = 10)$')
    
ax2 = fig.add_subplot(gs[1])
im2 = ax2.pcolor(xx, yy, f_values[:, :, 1], shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax2.set_xlabel(r'$x_1$', fontsize=16)
ax2.set_ylabel(r'$x_2$', fontsize=16)
ax2.set_title(r'True POC Surface $j = 2$')

cbar_ax = fig.add_subplot(gs[2])
fig.colorbar(im2, cax=cbar_ax)

pos = cbar_ax.get_position() 
cbar_ax.set_position([pos.x0 - 0.02, pos.y0, pos.width, pos.height]) 
    
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.3)
    
ax1 = fig.add_subplot(gs[0])
im1 = ax1.pcolor(xx, yy, monte_carlo_mean_surface_3_1[:, :, 2], shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax1.set_xlabel(r'$x_1$', fontsize=16)
ax1.set_ylabel(r'$x_2$', fontsize=16)
ax1.set_title(r'MC Mean KANE POC Surface ($j = 3,\,\theta = 1)$')

ax2 = fig.add_subplot(gs[1])
im2 = ax2.pcolor(xx, yy, monte_carlo_mean_surface_3_5[:, :, 2], shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax2.set_xlabel(r'$x_1$', fontsize=16)
ax2.set_ylabel(r'$x_2$', fontsize=16)
ax2.set_title(r'MC Mean KANE POC Surface ($j = 3,\,\theta = 5)$')

cbar_ax = fig.add_subplot(gs[2])
fig.colorbar(im2, cax=cbar_ax)

pos = cbar_ax.get_position() 
cbar_ax.set_position([pos.x0 - 0.02, pos.y0, pos.width, pos.height]) 

plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.3)
    
ax1 = fig.add_subplot(gs[0])
im1 = ax1.pcolor(xx, yy, monte_carlo_mean_surface_3_10[:, :, 2], shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax1.set_xlabel(r'$x_1$', fontsize=16)
ax1.set_ylabel(r'$x_2$', fontsize=16)
ax1.set_title(r'MC Mean KANE POC Surface ($j = 3,\,\theta = 10)$')
    
ax2 = fig.add_subplot(gs[1])
im2 = ax2.pcolor(xx, yy, f_values[:, :, 2], shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax2.set_xlabel(r'$x_1$', fontsize=16)
ax2.set_ylabel(r'$x_2$', fontsize=16)
ax2.set_title(r'True POC Surface $j = 3$')

cbar_ax = fig.add_subplot(gs[2])
fig.colorbar(im2, cax=cbar_ax)

pos = cbar_ax.get_position() 
cbar_ax.set_position([pos.x0 - 0.02, pos.y0, pos.width, pos.height]) 

plt.tight_layout()
plt.show()